Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
graph = tf.Graph()
with graph.as_default():
    # variable bash size
    tf_train_dataset = tf.placeholder(tf.float32,shape=[None, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[None, num_labels])
    # variable regularization parameter
    tf_p_regularization = tf.placeholder(tf.float32, shape=[])
    # variable learning rate
    tf_p_learning_rate = tf.placeholder(tf.float32, shape=[])
    # variable momentum
    # tf_p_momentum = tf.placeholder(tf.float32, shape=[])

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
  
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
            + tf_p_regularization * tf.nn.l2_loss(weights))
  
    optimizer = tf.train.GradientDescentOptimizer(tf_p_learning_rate).minimize(loss)
  
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [28]:
def train_mini_batch(dataset, labels, session, step, batch_size, regularization, learning_rate):
    offset = (step * batch_size) % (labels.shape[0] - batch_size)
    batch_data = dataset[offset:(offset + batch_size), :]
    batch_labels = labels[offset:(offset + batch_size), :]
    
    feed_dict = {
        tf_train_dataset : batch_data, 
        tf_train_labels : batch_labels, 
        tf_p_regularization : regularization,
        tf_p_learning_rate : learning_rate,
    }
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    acc = accuracy(predictions, batch_labels)
    return l, acc
    


batch_size = 128

regularization_parameters = [0, 0.001, 0.003, 0.01, 0.03, 0.1]
init_learning_rate = 0.5
decay = 0.8
nb_steps = 5000

accuracies = []

for regularization in regularization_parameters:    
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        
        step = 0
        learning_rate = init_learning_rate
        for step in range(nb_steps):
            l, acc = train_mini_batch(train_dataset, train_labels, session, step, batch_size, regularization, learning_rate)
            if (step % 500 == 0):
                print('Step {}, learning rate {:.2f}: minibatch loss = {:.2f}, minibatch accuracy = {:.2f}'
                      .format(step, learning_rate, l, acc))
                learning_rate *= decay
                
        acc = accuracy(valid_prediction.eval(), valid_labels)
        params = (batch_size, regularization, learning_rate, acc)
        print("Final validation accuracy with batch_size = {}, regularization parameter = {:.3f}, init_learning_rate = {:.2f} : {:.2f}\n"
              .format(batch_size, regularization, init_learning_rate, acc))
        accuracies.append(params)
        
best_regul = regularization_parameters[np.argmax(accuracies, axis=0)[3]]
print('Using best regul parameter ({})'.format(best_regul))

with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()    
        step = 0
        learning_rate = init_learning_rate
        for step in range(nb_steps):
            l, acc = train_mini_batch(train_dataset, train_labels, session, step, batch_size, best_regul, learning_rate)
            if (step % 500 == 0):
                print('Step {}, learning rate {:.2f}: minibatch loss = {:.2f}, minibatch accuracy = {:.2f}'
                      .format(step, learning_rate, l, acc))
                learning_rate *= decay
                
        acc = accuracy(test_prediction.eval(), test_labels)
        print("Final test accuracy with batch_size = {}, regularization parameter = {:.3f}, init_learning_rate = {:.2f} : {}\n"
              .format(batch_size, best_regul, init_learning_rate, acc))
    

Step 0, learning rate 0.50: minibatch loss = 14.23, minibatch accuracy = 13.28
Step 500, learning rate 0.40: minibatch loss = 1.41, minibatch accuracy = 76.56
Step 1000, learning rate 0.32: minibatch loss = 1.61, minibatch accuracy = 75.00
Step 1500, learning rate 0.26: minibatch loss = 1.17, minibatch accuracy = 78.12
Step 2000, learning rate 0.20: minibatch loss = 1.35, minibatch accuracy = 76.56
Step 2500, learning rate 0.16: minibatch loss = 1.12, minibatch accuracy = 75.00
Step 3000, learning rate 0.13: minibatch loss = 0.77, minibatch accuracy = 80.47
Step 3500, learning rate 0.10: minibatch loss = 1.36, minibatch accuracy = 71.88
Step 4000, learning rate 0.08: minibatch loss = 0.94, minibatch accuracy = 79.69
Step 4500, learning rate 0.07: minibatch loss = 0.71, minibatch accuracy = 89.84
Final validation accuracy with batch_size = 128, regularization parameter = 0.000, init_learning_rate = 0.50 : 78.27

Step 0, learning rate 0.50: minibatch loss = 21.19, minibatch accuracy = 15

In [29]:
batch_size = 128
num_relus = 1024

graph = tf.Graph()
with graph.as_default():
    # variable bash size
    tf_train_dataset = tf.placeholder(tf.float32,shape=[None, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[None, num_labels])
    # variable regularization parameter
    tf_p_regularization = tf.placeholder(tf.float32, shape=[])
    # variable learning rate
    tf_p_learning_rate = tf.placeholder(tf.float32, shape=[])
    # variable momentum
    # tf_p_momentum = tf.placeholder(tf.float32, shape=[])

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_relus]))   
    biases1 = tf.Variable(tf.zeros([num_relus]))
    weights2 = tf.Variable(tf.truncated_normal([num_relus, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    logits2 = tf.nn.relu(logits1)
    logits3 = tf.matmul(logits2, weights2) + biases2
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits3, tf_train_labels))
            + tf_p_regularization * tf.nn.l2_loss(weights1)
            + tf_p_regularization * tf.nn.l2_loss(weights2)
           )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(tf_p_learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits3)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [32]:
batch_size = 128

regularization_parameters = [0, 0.001, 0.003, 0.01, 0.03, 0.1]
init_learning_rate = 0.5
decay = 0.8
nb_steps = 3001

accuracies = []

for regularization in regularization_parameters:    
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        
        step = 0
        learning_rate = init_learning_rate
        for step in range(nb_steps):
            l, acc = train_mini_batch(train_dataset, train_labels, session, step, batch_size, regularization, learning_rate)
            if (step % 500 == 0):
                print('Step {}, learning rate {:.2f}: minibatch loss = {:.2f}, minibatch accuracy = {:.2f}'
                      .format(step, learning_rate, l, acc))
                learning_rate *= decay
                
        acc = accuracy(valid_prediction.eval(), valid_labels)
        params = (batch_size, regularization, learning_rate, acc)
        print("Final validation accuracy with batch_size = {}, regularization parameter = {:.3f}, init_learning_rate = {:.2f} : {:.2f}\n"
              .format(batch_size, regularization, init_learning_rate, acc))
        accuracies.append(params)
        
best_regul = regularization_parameters[np.argmax(accuracies, axis=0)[3]]
print('Using best regul parameter ({})'.format(best_regul))

with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()    
        step = 0
        learning_rate = init_learning_rate
        for step in range(nb_steps):
            l, acc = train_mini_batch(train_dataset, train_labels, session, step, batch_size, best_regul, learning_rate)
            if (step % 500 == 0):
                print('Step {}, learning rate {:.2f}: minibatch loss = {:.2f}, minibatch accuracy = {:.2f}'
                      .format(step, learning_rate, l, acc))
                learning_rate *= decay
                
        acc = accuracy(test_prediction.eval(), test_labels)
        print("Final test accuracy with batch_size = {}, regularization parameter = {:.3f}, init_learning_rate = {:.2f} : {}\n"
              .format(batch_size, best_regul, init_learning_rate, acc))
    

Step 0, learning rate 0.50: minibatch loss = 392.51, minibatch accuracy = 5.47
Step 500, learning rate 0.40: minibatch loss = 20.01, minibatch accuracy = 72.66
Step 1000, learning rate 0.32: minibatch loss = 14.04, minibatch accuracy = 82.81
Step 1500, learning rate 0.26: minibatch loss = 4.20, minibatch accuracy = 83.59
Step 2000, learning rate 0.20: minibatch loss = 7.80, minibatch accuracy = 84.38
Step 2500, learning rate 0.16: minibatch loss = 1.42, minibatch accuracy = 91.41
Step 3000, learning rate 0.13: minibatch loss = 0.99, minibatch accuracy = 85.94
Final validation accuracy with batch_size = 128, regularization parameter = 0.000, init_learning_rate = 0.50 : 83.61

Step 0, learning rate 0.50: minibatch loss = 619.33, minibatch accuracy = 18.75
Step 500, learning rate 0.40: minibatch loss = 228.81, minibatch accuracy = 76.56
Step 1000, learning rate 0.32: minibatch loss = 156.57, minibatch accuracy = 78.12
Step 1500, learning rate 0.26: minibatch loss = 116.23, minibatch accur

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [34]:
batch_size = 128

best_regul = 0.003
init_learning_rate = 0.5
decay = 0.8
nb_steps = 3001

accuracies = []

with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()    
        step = 0
        learning_rate = init_learning_rate
        for step in range(nb_steps):
            l, acc = train_mini_batch(train_dataset[:3*batch_size], train_labels[:3*batch_size], session, step, batch_size, best_regul, learning_rate)
            if (step % 500 == 0):
                print('Step {}, learning rate {:.2f}: minibatch loss = {:.2f}, minibatch accuracy = {:.2f}'
                      .format(step, learning_rate, l, acc))
                learning_rate *= decay
                
        acc = accuracy(test_prediction.eval(), test_labels)
        print("Final test accuracy with batch_size = {}, regularization parameter = {:.3f}, init_learning_rate = {:.2f} : {}\n"
              .format(batch_size, best_regul, init_learning_rate, acc))
    

Step 0, learning rate 0.50: minibatch loss = 9768.02, minibatch accuracy = 7.81
Step 500, learning rate 0.40: minibatch loss = 0.66, minibatch accuracy = 100.00
Step 1000, learning rate 0.32: minibatch loss = 0.55, minibatch accuracy = 100.00
Step 1500, learning rate 0.26: minibatch loss = 0.54, minibatch accuracy = 100.00
Step 2000, learning rate 0.20: minibatch loss = 0.53, minibatch accuracy = 100.00
Step 2500, learning rate 0.16: minibatch loss = 0.53, minibatch accuracy = 100.00
Step 3000, learning rate 0.13: minibatch loss = 0.53, minibatch accuracy = 100.00
Final test accuracy with batch_size = 128, regularization parameter = 0.030, init_learning_rate = 0.50 : 83.57



---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [39]:
batch_size = 128
num_relus = 1024

graph = tf.Graph()
with graph.as_default():
    # variable bash size
    tf_train_dataset = tf.placeholder(tf.float32,shape=[None, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, shape=[None, num_labels])
    # variable learning rate
    tf_p_learning_rate = tf.placeholder(tf.float32, shape=[])
    # variable momentum
    # tf_p_momentum = tf.placeholder(tf.float32, shape=[])

    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_relus]))   
    biases1 = tf.Variable(tf.zeros([num_relus]))
    weights2 = tf.Variable(tf.truncated_normal([num_relus, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    logits2 = tf.nn.relu(logits1)
    logits3 = tf.matmul(logits2, weights2) + biases2
    logits4 = tf.nn.dropout(logits3, 0.5)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits4, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(tf_p_learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits4)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [40]:
def train_mini_batch(dataset, labels, session, step, batch_size, learning_rate):
    offset = (step * batch_size) % (labels.shape[0] - batch_size)
    batch_data = dataset[offset:(offset + batch_size), :]
    batch_labels = labels[offset:(offset + batch_size), :]
    
    feed_dict = {
        tf_train_dataset : batch_data, 
        tf_train_labels : batch_labels, 
        tf_p_learning_rate : learning_rate,
    }
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    acc = accuracy(predictions, batch_labels)
    return l, acc

batch_size = 128

best_regul = 0.003
init_learning_rate = 0.5
decay = 0.8
nb_steps = 3001

accuracies = []

with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()    
        step = 0
        learning_rate = init_learning_rate
        for step in range(nb_steps):
            l, acc = train_mini_batch(train_dataset[:3*batch_size], train_labels[:3*batch_size], session, step, batch_size, learning_rate)
            if (step % 500 == 0):
                print('Step {}, learning rate {:.2f}: minibatch loss = {:.2f}, minibatch accuracy = {:.2f}'
                      .format(step, learning_rate, l, acc))
                learning_rate *= decay
                
        acc = accuracy(test_prediction.eval(), test_labels)
        print("Final test accuracy with batch_size = {}, dropout half, init_learning_rate = {:.2f} : {}\n"
              .format(batch_size, init_learning_rate, acc))
    

Step 0, learning rate 0.50: minibatch loss = 687.36, minibatch accuracy = 10.16
Step 500, learning rate 0.40: minibatch loss = 0.88, minibatch accuracy = 52.34
Step 1000, learning rate 0.32: minibatch loss = 0.94, minibatch accuracy = 50.78
Step 1500, learning rate 0.26: minibatch loss = 0.74, minibatch accuracy = 63.28
Step 2000, learning rate 0.20: minibatch loss = 0.82, minibatch accuracy = 60.94
Step 2500, learning rate 0.16: minibatch loss = 0.81, minibatch accuracy = 53.12
Step 3000, learning rate 0.13: minibatch loss = 0.75, minibatch accuracy = 61.72
Final test accuracy with batch_size = 128, dropout half, init_learning_rate = 0.50 : 78.71



---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
